In [1]:
import os

In [2]:
%pwd

'/Users/anishmitagar/Documents/repos/summer_2023_projects/end-to-end-diabetes-project/research'

In [3]:
os.chdir('/Users/anishmitagar/Documents/repos/summer_2023_projects/end-to-end-diabetes-project')

In [4]:
%pwd

'/Users/anishmitagar/Documents/repos/summer_2023_projects/end-to-end-diabetes-project'

In [5]:
from os.path import exists

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelValidationConfig:
    root_dir: Path
    save_dir: Path
    data_path: Path
    best_acc_model_path_pth: Path
    best_acc_model_path_pkl: Path
    best_f1_model_path_pth: Path
    best_f1_model_path_pkl: Path


In [7]:
from mlClassifier.constants import *
from mlClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_validation_config(self) -> ModelValidationConfig:
        config = self.config.model_validation
        

        create_directories([config.root_dir])

        model_validation_config = ModelValidationConfig(
            root_dir=config.root_dir,
            save_dir=config.save_dir,
            data_path=config.data_path,
            best_acc_model_path_pth=config.best_acc_model_path_pth,
            best_acc_model_path_pkl=config.best_acc_model_path_pkl,
            best_f1_model_path_pth=config.best_f1_model_path_pth,
            best_f1_model_path_pkl=config.best_f1_model_path_pkl
        )

        return model_validation_config

In [9]:
config = ConfigurationManager()
model_trainer_config = config.get_model_validation_config()

[2023-07-22 10:36:45,811: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-22 10:36:45,813: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-22 10:36:45,814: INFO: common: created directory at: artifacts]
[2023-07-22 10:36:45,814: INFO: common: created directory at: artifacts/model_validation]


In [10]:
import pandas as pd
from os.path import exists
import os
import sys
import dill
import pickle
import torch

def save_object(file_path, obj):

    dir_path = os.path.dirname(file_path)

    os.makedirs(dir_path, exist_ok=True)

    with open(file_path, "wb") as file_obj:
        dill.dump(obj, file_obj)
    

    
def load_object(file_path):

    with open(file_path, "rb") as file_obj:
        return pickle.load(file_obj)


In [11]:
import torch.nn as nn

class Deep(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(15, 15)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(15, 15)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(15, 15)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(15, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [12]:
from sklearn import metrics  

In [13]:
class ModelValidation:
    def __init__(self, config: ModelValidationConfig):
        self.config = config

    def validate_model_performance(self):
        df = pd.read_csv(self.config.data_path)
        X, y = df.drop(columns=['diabetes'],axis=1), df['diabetes']
        X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32)
        y_tensor = torch.tensor(y.to_numpy(), dtype=torch.float32).reshape(-1, 1)

        results = {"Best_Accuracy_Model_Metrics": None, "Best_F1_Model_Metrics": None}

        if exists(self.config.best_acc_model_path_pkl):
            model = load_object(self.config.best_acc_model_path_pkl)
            y_pred = model.predict(X)
            acc, f1, precision, recall = metrics.accuracy_score(y, y_pred), metrics.f1_score(y, y_pred), metrics.precision_score(y, y_pred), metrics.recall_score(y, y_pred)
            results['Best_Accuracy_Model_Metrics'] = {"Accuracy_Score": acc, "F1_Score": f1, "Precision_Score": precision, "Recall_Score": recall}
        elif exists(self.config.best_acc_model_path_pth):
            model = Deep()
            model.load_state_dict(torch.load(self.config.best_acc_model_path_pth))
            model.eval()
            y_pred_tensor = model(X_tensor)
            y_pred_tensor = y_pred_tensor.round().detach().cpu().numpy()
            y_tensor_np = y_tensor.numpy()
            acc, f1, precision, recall = metrics.accuracy_score(y_tensor_np, y_pred_tensor), metrics.f1_score(y_tensor_np, y_pred_tensor), metrics.precision_score(y_tensor_np, y_pred_tensor), metrics.recall_score(y_tensor_np, y_pred_tensor)
            results['Best_Accuracy_Model_Metrics'] = {"Accuracy_Score": acc, "F1_Score": f1, "Precision_Score": precision, "Recall_Score": recall}
        else:
            print("Error")

        if exists(self.config.best_f1_model_path_pkl):
            model = load_object(self.config.best_f1_model_path_pkl)
            y_pred = model.predict(X)
            acc, f1, precision, recall = metrics.accuracy_score(y, y_pred), metrics.f1_score(y, y_pred), metrics.precision_score(y, y_pred), metrics.recall_score(y, y_pred)
            results['Best_F1_Model_Metrics'] = {"Accuracy_Score": acc, "F1_Score": f1, "Precision_Score": precision, "Recall_Score": recall}
        elif exists(self.config.best_f1_model_path_pth):
            model = Deep()
            model.load_state_dict(torch.load(self.config.best_f1_model_path_pth))
            model.eval()
            y_pred_tensor = model(X_tensor)
            y_pred_tensor = y_pred_tensor.round().detach().cpu().numpy()
            y_tensor_np = y_tensor.numpy() 
            acc, f1, precision, recall = metrics.accuracy_score(y_tensor_np, y_pred_tensor), metrics.f1_score(y_tensor_np, y_pred_tensor), metrics.precision_score(y_tensor_np, y_pred_tensor), metrics.recall_score(y_tensor_np, y_pred_tensor)
            results['Best_F1_Model_Metrics'] = {"Accuracy_Score": acc, "F1_Score": f1, "Precision_Score": precision, "Recall_Score": recall}
        else:
            print("Error")

        save_json(path = Path(self.config.save_dir), data = results)

In [14]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_model_validation_config()
    data_transformation = ModelValidation(config=data_validation_config)
    data_transformation.validate_model_performance()
except Exception as e:
    raise e

[2023-07-22 10:36:46,962: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-22 10:36:46,965: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-22 10:36:46,965: INFO: common: created directory at: artifacts]
[2023-07-22 10:36:46,966: INFO: common: created directory at: artifacts/model_validation]
[2023-07-22 10:36:47,672: INFO: common: json file saved at: artifacts/model_validation/results.json]
